In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pickle
from scipy.sparse.linalg import svds, eigs

import warnings
warnings.filterwarnings('ignore')

### Reading Data

In [0]:
train_graph = nx.read_edgelist('/content/gdrive/My Drive/Colab Notebooks/Facebook/after eda/train_pos.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)

In [0]:
print(nx.info(train_graph))

Name: 
Type: DiGraph
Number of nodes: 1780722
Number of edges: 7550015
Average in degree:   4.2399
Average out degree:   4.2399


### Jaccard Similarity

In [0]:
## jaccarad for followee
def jaccard(a,b):
    try:
        if len(train_graph.successors(a)) == 0 | len(train_graph.successors(b)) == 0:
            return 0 
        sim = len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b))))/len(set(train_graph.successors(a)).union(train_graph.successors(b)))
        
    except:
        return 0
    return sim

In [0]:
## jaccard for followers

def jaccard_followers(a,b):
    try:
        if len(train_graph.predecessors(a)) == 0 | len(train_graph.predecessors(b)) == 0:
            return 0 
        sim = len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b))))/len(set(train_graph.predecessors(a)).union(train_graph.predecessors(b)))
        
    except:
        return 0
    return sim


### Cosine Distance


In [0]:
## cosine for followee

def cosine(a,b):
    try:
        if len(train_graph.successors(a)) == 0 | len(train_graph.successors(b)) == 0:
            return 0 
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                    (math.sqrt(len(set(train_graph.successors(a)))*len((set(train_graph.successors(b))))))
        
    except:
        return 0
    return sim


In [0]:
## cosine for followers

def cosine_followers(a,b):
    try:
        if len(train_graph.predecessors(a)) == 0 | len(train_graph.predecessors(b)) == 0:
            return 0 
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                    (math.sqrt(len(set(train_graph.predecessors(a)))*len((set(train_graph.predecessors(b))))))
        
    except:
        return 0
    return sim


## Page Rank

In [0]:
pr = nx.pagerank(train_graph,alpha=0.85)
pickle.dump(pr,open('/content/gdrive/My Drive/Colab Notebooks/Facebook/after eda/pagerank.p','wb'))

In [0]:
pr = pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/Facebook/after eda/pagerank.p','rb'))

In [0]:
print('min',pr[min(pr, key=pr.get)])
print('max',pr[max(pr, key=pr.get)])
print('mean',float(sum(pr.values())) / len(pr))
mean_pr = float(sum(pr.values())) / len(pr)

min 1.6556497245737814e-07
max 2.7098251341935827e-05
mean 5.615699699389075e-07


5.615699699389075e-07

In [0]:
# now if there are some nodes which are not in test data but are in train data 
# then we will use mean pagerank values for them.

## Shortest Path 

we are finding shortest node between two nodes and if they has direct edge between then we will remove that edge and calculate shortest path


In [0]:
def short_path(a,b):
  try:
    if train_graph.has_edge(a,b):
      train_graph.remove_edge(a,b)
      p = nx.shortest_path_length(train_graph,source=a,target=b)
      train_graph.add_edge(a,b)
    else:
      p = nx.shortest_path_length(train_graph,source=a,target=b)
    return p 
  except:
      return -1

In [0]:
#testing
short_path(1,100)

9

In [0]:
#testing
short_path(669354,1635354)

-1

## Checking for same community

In [0]:
#getting weekly connected edges from graph 
wcc=list(nx.weakly_connected_components(train_graph))

In [0]:
def belongs_to_same_wcc(a,b):
    index = []
    if train_graph.has_edge(b,a):
        return 1
    if train_graph.has_edge(a,b):
            for i in wcc:
                if a in i:
                    index= i
                    break
            if (b in index):
                train_graph.remove_edge(a,b)
                if short_path(a,b)==-1:
                    train_graph.add_edge(a,b)
                    return 0
                else:
                    train_graph.add_edge(a,b)
                    return 1
            else:
                return 0
    else:
            for i in wcc:
                if a in i:
                    index= i
                    break
            if(b in index):
                return 1
            else:
                return 0

In [0]:
belongs_to_same_wcc(1,101231)

1

In [0]:
belongs_to_same_wcc(1,100)

1

## Adar Index



In [0]:
def adar(a,b):
  try:
    n= list(set(train_graph.successors(a).intersection(set(train_graph.successors(b)))))
    if len(n)!=0:
      for i in n:
        sum = sum(1+np.log10(len(list(train_graph.predecessors(i)))))
      return sum
    else:
      return 0
  except:
      return 0

In [0]:
adar(1,102141)

0

#### Is person following back?or going to follow back?


In [0]:
def follow_back(a,b):
  if train_graph.has_edge(b,a):
    return 1
  else:
    return 0

## Katz Centrality

In [0]:
katz = nx.katz.katz_centrality(train_graph,alpha=0.005,beta=1)
pickle.dump(katz,open('/content/gdrive/My Drive/Colab Notebooks/Facebook/after eda/katz.p','wb'))

In [0]:
katz = pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/Facebook/after eda/katz.p','rb'))

In [0]:
print(f'min {min(katz.values())}')
print(f'max {max(katz.values())}')
print(f'mean {float(sum(katz.values())/len(katz))}')

mean_katz = float(sum(katz.values())/len(katz))

min 0.0007313532484065916
max 0.003394554981699122
mean 0.0007483800935562018


## HITS Score


In [0]:
hits = nx.hits(train_graph,max_iter=100,normalized=True)

In [0]:
pickle.dump(hits,open('/content/gdrive/My Drive/Colab Notebooks/Facebook/after eda/hits.p','wb'))

In [0]:
hits = pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/Facebook/after eda/hits.p','rb'))

## Creating some other basic feature 

1.Num followers for source and destination node

 
2.Num of followee for source and destination node


In [0]:
def compute_features_stage1(df_final):
    #calculating no of followers followees for source and destination
    #calculating intersection of followers and followees for source and destination
    num_followers_s=[]
    num_followees_s=[]
    num_followers_d=[]
    num_followees_d=[]
    inter_followers=[]
    inter_followees=[]
    for i,row in df_final.iterrows():
        try:
            s1=set(train_graph.predecessors(row['source_node']))
            s2=set(train_graph.successors(row['source_node']))
        except:
            s1 = set()
            s2 = set()
        try:
            d1=set(train_graph.predecessors(row['destination_node']))
            d2=set(train_graph.successors(row['destination_node']))
        except:
            d1 = set()
            d2 = set()
        num_followers_s.append(len(s1))
        num_followees_s.append(len(s2))

        num_followers_d.append(len(d1))
        num_followees_d.append(len(d2))

        inter_followers.append(len(s1.intersection(d1)))
        inter_followees.append(len(s2.intersection(d2)))
    
    return num_followers_s, num_followers_d, num_followees_s, num_followees_d, inter_followers, inter_followees

## Reading sample form train and test data


In [0]:
## to take sample from train data
import random
Train_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Facebook/after eda/train_after_eda.csv')
n_train = len(Train_data)
sample_size = 100000

skip_train = sorted(random.sample(range(1,n_train+1),n_train-sample_size))

In [0]:
## to take sample from test data
import random
Test_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Facebook/after eda/test_after_eda.csv')
n_test = len(Test_data)
sample_size = 50000

skip_test = sorted(random.sample(range(1,n_test+1),n_test-sample_size))

In [0]:
print(f'len of train data {len(Train_data)}')
print(f'Number of rows we are going to eliminate {len(skip_train)}')

print('='*80)
print(f'len of train data {len(Test_data)}')
print(f'Number of rows we are going to eliminate {len(skip_test)}')

len of train data 15100029
Number of rows we are going to eliminate 15000029
len of train data 3775007
Number of rows we are going to eliminate 3725007


In [0]:
df_final_train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Facebook/after eda/train_after_eda.csv', skiprows=skip_train, names=['source_node', 'destination_node'])
df_final_train['indicator_link'] = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Facebook/after eda/train_y.csv', skiprows=skip_train, names=['indicator_link'])
print("Our train matrix size ",df_final_train.shape)
df_final_train.head(2)

Our train matrix size  (100001, 3)


,source_node,destination_node,indicator_link
0,273084,1505602,1
1,1443670,271786,1


In [0]:
df_final_test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Facebook/after eda/test_after_eda.csv', skiprows=skip_test, names=['source_node', 'destination_node'])
df_final_test['indicator_link'] = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Facebook/after eda/test_y.csv', skiprows=skip_test, names=['indicator_link'])
print("Our test matrix size ",df_final_test.shape)
df_final_test.head(2)

Our test matrix size  (50001, 3)


,source_node,destination_node,indicator_link
0,848424,784690,1
1,783368,1420922,1


### Add Features to data

In [0]:
## Jaccrad followee
df_final_train['jaccard_followee'] = df_final_train.apply(lambda row: jaccard(row['source_node'],row['destination_node']),axis=1)

df_final_test['jaccard_followee'] = df_final_test.apply(lambda row: jaccard(row['source_node'],row['destination_node']),axis=1)

## Jaccard followers
df_final_train['jaccard_followers'] = df_final_train.apply(lambda row: jaccard_followers(row['source_node'],row['destination_node']),axis=1)

df_final_test['jaccard_followers'] = df_final_test.apply(lambda row: jaccard_followers(row['source_node'],row['destination_node']),axis=1)

#cosine followee

df_final_train['cosine_followee'] = df_final_train.apply(lambda row: cosine(row['source_node'],row['destination_node']),axis=1)

df_final_test['cosine_followee'] = df_final_test.apply(lambda row: cosine(row['source_node'],row['destination_node']),axis=1)

#cosine followers

df_final_train['cosine_followers'] = df_final_train.apply(lambda row: cosine_followers(row['source_node'],row['destination_node']),axis=1)

df_final_test['cosine_followers'] = df_final_test.apply(lambda row: cosine_followers(row['source_node'],row['destination_node']),axis=1)

In [0]:
# compute basic feature for train
df_final_train['num_followers_s'],df_final_train['num_followers_d'],\
df_final_train['num_followees_s'],df_final_train['num_followees_d'],\
df_final_train['inter_followers'],df_final_train['inter_followees'] = compute_features_stage1(df_final_train)

# compute basic feature for test
df_final_test['num_followers_s'],df_final_test['num_followers_d'],\
df_final_test['num_followees_s'],df_final_test['num_followees_d'],\
df_final_test['inter_followers'],df_final_test['inter_followees'] = compute_features_stage1(df_final_test)

### Save the data (stage 1)

In [0]:
hdf = HDFStore('/content/gdrive/My Drive/Colab Notebooks/Facebook/HDF_files/sample_stage1.h5')
hdf.put('train_df',df_final_train,format='table',data_columns=True)
hdf.put('test_df',df_final_test,format='table',data_columns=True)
hdf.close()

In [0]:
## to read from hdf 
from pandas import read_hdf
df_final_train = read_hdf('/content/gdrive/My Drive/Colab Notebooks/Facebook/HDF_files/sample_stage1.h5', 'train_df',mode='r')
df_final_test = read_hdf('/content/gdrive/My Drive/Colab Notebooks/Facebook/HDF_files/sample_stage1.h5','test_df',mode='r')

### Stage 2

In [0]:
#mapping adar index on train
df_final_train['adar_index'] = df_final_train.apply(lambda row: adar(row['source_node'],row['destination_node']),axis=1)
    #mapping adar index on test
df_final_test['adar_index'] = df_final_test.apply(lambda row: adar(row['source_node'],row['destination_node']),axis=1)

    #--------------------------------------------------------------------------------------------------------
    #mapping followback or not on train
df_final_train['follows_back'] = df_final_train.apply(lambda row: follow_back(row['source_node'],row['destination_node']),axis=1)

    #mapping followback or not on test
df_final_test['follows_back'] = df_final_test.apply(lambda row: follow_back(row['source_node'],row['destination_node']),axis=1)

    #--------------------------------------------------------------------------------------------------------
    #mapping same component of wcc or not on train
df_final_train['same_comp'] = df_final_train.apply(lambda row: belongs_to_same_wcc(row['source_node'],row['destination_node']),axis=1)

    ##mapping same component of wcc or not on train
df_final_test['same_comp'] = df_final_test.apply(lambda row: belongs_to_same_wcc(row['source_node'],row['destination_node']),axis=1)
    
    #--------------------------------------------------------------------------------------------------------
    #mapping shortest path on train 
df_final_train['shortest_path'] = df_final_train.apply(lambda row: short_path(row['source_node'],row['destination_node']),axis=1)
    #mapping shortest path on test
df_final_test['shortest_path'] = df_final_test.apply(lambda row: short_path(row['source_node'],row['destination_node']),axis=1)   

In [0]:
hdf = pd.HDFStore('/content/gdrive/My Drive/Colab Notebooks/Facebook/HDF_files/sample_stage2.h5')
hdf.put('train_df',df_final_train, format='table', data_columns=True)
hdf.put('test_df',df_final_test, format='table', data_columns=True)
hdf.close() 

In [0]:
df_final_train = read_hdf('/content/gdrive/My Drive/Colab Notebooks/Facebook/HDF_files/sample_stage2.h5', 'train_df',mode='r')
df_final_test = read_hdf('/content/gdrive/My Drive/Colab Notebooks/Facebook/HDF_files/sample_stage2.h5', 'test_df',mode='r')

## Stage 3


In [0]:
## Page Rank
### passing a default value mean_pr if there is nothing in graph
df_final_train['page_rank_s'] = df_final_train.source_node.apply(lambda x: pr.get(x,mean_pr))
df_final_train['page_rank_d'] = df_final_train.destination_node.apply(lambda x: pr.get(x,mean_pr))

df_final_test['page_rank_s'] = df_final_test.source_node.apply(lambda x: pr.get(x,mean_pr))
df_final_test['page_rank_d'] = df_final_test.destination_node.apply(lambda x: pr.get(x,mean_pr))
############################################################################################################

### Katz Centrality
df_final_train['katz_s'] = df_final_train.source_node.apply(lambda x: katz.get(x,mean_katz))
df_final_train['katz_d'] = df_final_train.destination_node.apply(lambda x: katz.get(x,mean_katz))

df_final_test['katz_s'] = df_final_test.source_node.apply(lambda x: katz.get(x,mean_katz))
df_final_test['katz_d'] = df_final_test.destination_node.apply(lambda x: katz.get(x,mean_katz))

############################################################################################################

## Hits
### if there is nothing in train graph adding zero
df_final_train['hubs_s'] = df_final_train.source_node.apply(lambda x: hits[0].get(x,0))
df_final_train['hubs_d'] = df_final_train.destination_node.apply(lambda x: hits[0].get(x,0))

df_final_test['hubs_s'] = df_final_test.source_node.apply(lambda x: hits[0].get(x,0))
df_final_test['hubs_d'] = df_final_test.destination_node.apply(lambda x: hits[0].get(x,0))

############################################################################################################

df_final_train['autho_s'] = df_final_train.source_node.apply(lambda x: hits[1].get(x,0))
df_final_train['autho_d'] = df_final_train.destination_node.apply(lambda x: hits[1].get(x,0))

df_final_test['autho_s'] = df_final_test.source_node.apply(lambda x: hits[1].get(x,0))
df_final_test['autho_d'] = df_final_test.destination_node.apply(lambda x: hits[1].get(x,0))

In [0]:
## save the data 
hdf = pd.HDFStore('/content/gdrive/My Drive/Colab Notebooks/Facebook/HDF_files/sample_stage3.h5')
hdf.put('train_df',df_final_train,format='table',data_columns=True)
hdf.put('test_df',df_final_test,format='table',data_columns=True)
hdf.close()

In [0]:
df_final_train = pd.read_hdf('/content/gdrive/My Drive/Colab Notebooks/Facebook/HDF_files/sample_stage3.h5', 'train_df',mode='r')
df_final_test = pd.read_hdf('/content/gdrive/My Drive/Colab Notebooks/Facebook/HDF_files/sample_stage3.h5', 'test_df',mode='r')

## Weight Feature


1.   weight of incoming edges
2.   weight of outgoing edges
3.   weight of incoming edges + weight of outgoing edges
4.   weight of incoming edges * weight of outgoing edges
5.   2*weight of incoming edges + weight of outgoing edges
6.   weight of incoming edges + 2*weight of outgoing edges




In [0]:
from tqdm import tqdm
w_in = {}
w_out = {}

for i in tqdm(train_graph.nodes()):
  s1 = set(train_graph.predecessors(i))
  weight = 1.0/np.sqrt(1+len(s1))
  w_in[i] = weight

  s2 = set(train_graph.successors(i))
  weight2 = 1.0/np.sqrt(1+len(s2))
  w_out[i] = weight2

mean_in = np.mean(list(w_in.values()))
mean_out = np.mean(list(w_out.values()))

100%|██████████| 1780722/1780722 [00:12<00:00, 138306.37it/s]


In [0]:
#mapping to pandas train
df_final_train['weight_in'] = df_final_train.destination_node.apply(lambda x: w_in.get(x,mean_in))
df_final_train['weight_out'] = df_final_train.source_node.apply(lambda x: w_out.get(x,mean_out))

#mapping to pandas test
df_final_test['weight_in'] = df_final_test.destination_node.apply(lambda x: w_in.get(x,mean_in))
df_final_test['weight_out'] = df_final_test.source_node.apply(lambda x: w_out.get(x,mean_out))


    #some features engineerings on the in and out weights
df_final_train['weight_f1'] = df_final_train.weight_in + df_final_train.weight_out
df_final_train['weight_f2'] = df_final_train.weight_in * df_final_train.weight_out
df_final_train['weight_f3'] = (2*df_final_train.weight_in + 1*df_final_train.weight_out)
df_final_train['weight_f4'] = (1*df_final_train.weight_in + 2*df_final_train.weight_out)

    #some features engineerings on the in and out weights
df_final_test['weight_f1'] = df_final_test.weight_in + df_final_test.weight_out
df_final_test['weight_f2'] = df_final_test.weight_in * df_final_test.weight_out
df_final_test['weight_f3'] = (2*df_final_test.weight_in + 1*df_final_test.weight_out)
df_final_test['weight_f4'] = (1*df_final_test.weight_in + 2*df_final_test.weight_out)

## SVD

In [0]:
def svd(x, S):
    try:
        z = sadj_dict[x]
        return S[z]
    except:
        return [0,0,0,0,0,0]

In [0]:
#for svd features to get feature vector creating a dict node val and inedx in svd vector
sadj_col = sorted(train_graph.nodes())
sadj_dict = { val:idx for idx,val in enumerate(sadj_col)}

In [0]:
Adj = nx.adjacency_matrix(train_graph,nodelist=sorted(train_graph.nodes())).asfptype()

In [0]:
U, s, V = svds(Adj, k = 6)
print('Adjacency matrix Shape',Adj.shape)
print('U Shape',U.shape)
print('V Shape',V.shape)
print('s Shape',s.shape)

Adjacency matrix Shape (1780722, 1780722)
U Shape (1780722, 6)
V Shape (6, 1780722)
s Shape (6,)


In [0]:
import os
if not os.path.isfile('/content/gdrive/My Drive/Colab Notebooks/Facebook/HDF_files/sample_stage4.h5'):
    #===================================================================================================
    
    df_final_train[['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6']] = \
    df_final_train.source_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    
    df_final_train[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6']] = \
    df_final_train.destination_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    #===================================================================================================
    
    df_final_train[['svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6',]] = \
    df_final_train.source_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)

    df_final_train[['svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6']] = \
    df_final_train.destination_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)
    #===================================================================================================
    
    df_final_test[['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6']] = \
    df_final_test.source_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    
    df_final_test[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6']] = \
    df_final_test.destination_node.apply(lambda x: svd(x, U)).apply(pd.Series)

    #===================================================================================================
    
    df_final_test[['svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6',]] = \
    df_final_test.source_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)

    df_final_test[['svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6']] = \
    df_final_test.destination_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)
    #===================================================================================================

    hdf = pd.HDFStore('/content/gdrive/My Drive/Colab Notebooks/Facebook/HDF_files/sample_stage4.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.put('test_df',df_final_test, format='table', data_columns=True)
    hdf.close()

In [0]:
df_final_train = pd.read_hdf('/content/gdrive/My Drive/Colab Notebooks/Facebook/HDF_files/sample_stage4.h5', 'train_df',mode='r')
df_final_test = pd.read_hdf('/content/gdrive/My Drive/Colab Notebooks/Facebook/HDF_files/sample_stage4.h5', 'test_df',mode='r')

### Preferential Attachment

In [0]:
def pre_followee(a,b):
  try:

    if (set(len(train_graph.successors(a)))==0 | set(len(train_graph.successors(b))==0)):
      return 0
    sim = set(len(train_graph.successors(a))) * set(len(train_graph.successors(b)))
  except:
    return 0
  return sim

In [0]:
def pre_followers(a,b):
  try:

    if (set(len(train_graph.predecessors(a)))==0 | set(len(train_graph.predecessors(b))==0)):
      return 0
    sim = set(len(train_graph.predecessors(a))) * set(len(train_graph.predecessors(b)))
  except:
    return 0
  return sim

### Preferential Attachment for followee

In [0]:
df_final_train['pre_followee'] = df_final_train.apply(lambda row: pre_followee(row['source_node'],row['destination_node']),axis=1)

df_final_test['pre_followee'] = df_final_test.apply(lambda row: pre_followee(row['source_node'],row['destination_node']),axis=1)

### Preferential Attachment for followers

In [0]:
df_final_train['pre_follower'] = df_final_train.apply(lambda row: pre_followers(row['source_node'],row['destination_node']),axis=1)

df_final_test['pre_follower'] = df_final_test.apply(lambda row: pre_followers(row['source_node'],row['destination_node']),axis=1)

SVD DOT


In [0]:
def combine_svd(u1,u2,u3,u4,u5,u6,v1,v2,v3,v4,v5,v6):
  return np.array([u1,u2,u3,u4,u5,u6,v1,v2,v3,v4,v5,v6])

In [0]:
def svd_dot(a,b):
  return np.dot(a,b)

In [0]:
# Train Data

svd_dot_train = []
for i in range(len(df_final_train)):
  #source node
  svd_s = combine_svd(df_final_train.svd_u_s_1[i],df_final_train.svd_u_s_2[i],df_final_train.svd_u_s_3[i],df_final_train.svd_u_s_4[i],
                      df_final_train.svd_u_s_5[i],df_final_train.svd_u_s_6[i],df_final_train.svd_v_s_1[i],df_final_train.svd_v_s_2[i],df_final_train.svd_v_s_3[i],
                      df_final_train.svd_v_s_4[i],df_final_train.svd_v_s_5[i],df_final_train.svd_v_s_6[i])
  
  #destination node

  svd_d = combine_svd(df_final_train.svd_u_d_1[i],df_final_train.svd_u_d_2[i],df_final_train.svd_u_d_3[i],df_final_train.svd_u_d_4[i],
                      df_final_train.svd_u_d_5[i],df_final_train.svd_u_d_6[i],df_final_train.svd_v_d_1[i],df_final_train.svd_v_d_2[i],df_final_train.svd_v_d_3[i],
                      df_final_train.svd_v_d_4[i],df_final_train.svd_v_d_5[i],df_final_train.svd_v_d_6[i])
  
  svd_dot_train.append(svd_dot(svd_s,svd_d))

df_final_train['svd_dot'] = svd_dot_train


In [0]:
# Test Data

svd_dot_test = []
for i in range(len(df_final_test)):
  #source node
  svd_s = combine_svd(df_final_test.svd_u_s_1[i],df_final_test.svd_u_s_2[i],df_final_test.svd_u_s_3[i],df_final_test.svd_u_s_4[i],
                      df_final_test.svd_u_s_5[i],df_final_test.svd_u_s_6[i],df_final_test.svd_v_s_1[i],df_final_test.svd_v_s_2[i],df_final_test.svd_v_s_3[i],
                      df_final_test.svd_v_s_4[i],df_final_test.svd_v_s_5[i],df_final_test.svd_v_s_6[i])
  
  #destination node

  svd_d = combine_svd(df_final_test.svd_u_d_1[i],df_final_test.svd_u_d_2[i],df_final_test.svd_u_d_3[i],df_final_test.svd_u_d_4[i],
                      df_final_test.svd_u_d_5[i],df_final_test.svd_u_d_6[i],df_final_test.svd_v_d_1[i],df_final_test.svd_v_d_2[i],df_final_test.svd_v_d_3[i],
                      df_final_test.svd_v_d_4[i],df_final_test.svd_v_d_5[i],df_final_test.svd_v_d_6[i])
  
  svd_dot_test.append(svd_dot(svd_s,svd_d))

df_final_test['svd_dot'] = svd_dot_test


In [0]:
hdf = pd.HDFStore('/content/gdrive/My Drive/Colab Notebooks/Facebook/HDF_files/final_stage.h5')
hdf.put('train_df',df_final_train, format='table', data_columns=True)
hdf.put('test_df',df_final_test, format='table', data_columns=True)
hdf.close()